#### garrett r peternel

- _use spylon-kernel to enable scala api_
- _use magic function **%%python** to enable python api_

### create session

In [1]:
import org.apache.spark.sql.SparkSession

val spark = (SparkSession
.builder()
.appName("demos")
.master("local[8]") // use 8 cores in local mode on single machine for testing purposes only
.getOrCreate())

Intitializing Scala interpreter ...

Spark Web UI available at http://AERXLG9014XCC.us.lmco.com:4041
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1595434914648)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@da8cbd2


### json string

In [2]:
val jsonString = """
[
	{
		"game": "Destiny 1",
		"nest" : [
			
			{
				"release": "d1",
				"universe": {
					"class": "Titan",
					"characteristics": {
						"subclass": ["Defender"],
						"super": ["Ward of Dawn"]
					}
				}
			},

			{
				"release": "d1",
				"universe": {
					"class": "Hunter",
					"characteristics": {
						"subclass": ["Bladedancer"],
						"super": ["Arc Blade"]
					}
				}
			},

			{
				"release": "d1",
				"universe": {
					"class": "Warlock",
					"characteristics": {
						"subclass": ["Sunsigner"],
						"super": ["Radiance"]
					}
				}
			}
		]
	},

	{
		"game": "Destiny 2",
		"nest" : [
			
			{
				"release": "d2",
				"universe": {
					"class": "Titan",
					"characteristics": {
						"subclass": ["Sunbreaker", "Sentinel", "Striker"],
						"super": ["Hammer of Sol", "Sentinel Shield", "Fist of Havoc"]
					}
				}
			},

			{
				"release": "d2",
				"universe": {
					"class": "Hunter",
					"characteristics": {
						"subclass": ["Gunslinger", "Nightstalker", "Arcstrider"],
						"super": ["Golden Gun", "Shadowshot", "Arc Staff"]
					}
				}
			},

			{
				"release": "d2",
				"universe": {
					"class": "Warlock",
					"characteristics": {
						"subclass": ["Dawnblade", "Voidwalker", "Stormcaller"],
						"super": ["Daybreak", "Nova Bomb", "Stormtrance"]
					}
				}
			}
		]
	}
]
"""

jsonString: String =
"
[
	{
		"game": "Destiny 1",
		"nest" : [

			{
				"release": "d1",
				"universe": {
					"class": "Titan",
					"characteristics": {
						"subclass": ["Defender"],
						"super": ["Ward of Dawn"]
					}
				}
			},

			{
				"release": "d1",
				"universe": {
					"class": "Hunter",
					"characteristics": {
						"subclass": ["Bladedancer"],
						"super": ["Arc Blade"]
					}
				}
			},

			{
				"release": "d1",
				"universe": {
					"class": "Warlock",
					"characteristics": {
						"subclass": ["Sunsigner"],
						"super": ["Radiance"]
					}
				}
			}
		]
	},

	{
		"game": "Destiny 2",
		"nest" : [

			{
				"release": "d2",
				"universe": {
					"class": "Titan",
					"characteristics": {
						"sub...

### schema

In [3]:
import org.apache.spark.sql.types.{StructType, StringType, ArrayType}

val schema = (new StructType()
              .add("game", StringType, true)
              .add("nest", ArrayType(new StructType()
                                     .add("release", StringType, true)
                                     .add("universe", new StructType()
                                          .add("class", StringType, true)
                                          .add("characteristics", new StructType()
                                               .add("subclass", ArrayType(StringType))
                                               .add("super", ArrayType(StringType))
                                              )
                                          )
                                     )
                   )
              )


import org.apache.spark.sql.types.{StructType, StringType, ArrayType}
schema: org.apache.spark.sql.types.StructType = StructType(StructField(game,StringType,true), StructField(nest,ArrayType(StructType(StructField(release,StringType,true), StructField(universe,StructType(StructField(class,StringType,true), StructField(characteristics,StructType(StructField(subclass,ArrayType(StringType,true),true), StructField(super,ArrayType(StringType,true),true)),true)),true)),true),true))


### convert json string to rdd

In [4]:
val rdd = sc.parallelize(Seq(jsonString))

rdd: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at <console>:29


### convert rdd to dataframe

In [5]:
val nestedDf = (spark
                .read
                .option("multiline", "true")
                .schema(schema)
                .json(rdd)
               )

nestedDf: org.apache.spark.sql.DataFrame = [game: string, nest: array<struct<release:string,universe:struct<class:string,characteristics:struct<subclass:array<string>,super:array<string>>>>>]


In [6]:
nestedDf.printSchema
nestedDf.show(false)

root
 |-- game: string (nullable = true)
 |-- nest: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- release: string (nullable = true)
 |    |    |-- universe: struct (nullable = true)
 |    |    |    |-- class: string (nullable = true)
 |    |    |    |-- characteristics: struct (nullable = true)
 |    |    |    |    |-- subclass: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- super: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|game     |nest                                                                                                

### transform

In [7]:
import org.apache.spark.sql.functions.{explode}

val explodeRowsDf = nestedDf.select($"game", explode($"nest").as("row"))

val dotDf = (explodeRowsDf
             .select($"game",
                     $"row.release",
                     $"row.universe.*",
                     $"row.universe.characteristics.*")
             .drop("characteristics")
            )

import org.apache.spark.sql.functions.{arrays_zip}

val zipArraysDf = (dotDf
                   .select($"*", explode(arrays_zip($"subclass", $"super"))
                           .alias("arrayMap"))
                   .drop("subclass", "super")
                   )

val finalDf = (zipArraysDf
               .selectExpr("game",
                           "release",
                           "class",
                           "arrayMap.subclass",
                           "arrayMap.super")
               .orderBy($"class", $"game")
               )

import org.apache.spark.sql.functions.explode
explodeRowsDf: org.apache.spark.sql.DataFrame = [game: string, row: struct<release: string, universe: struct<class: string, characteristics: struct<subclass: array<string>, super: array<string>>>>]
dotDf: org.apache.spark.sql.DataFrame = [game: string, release: string ... 3 more fields]
import org.apache.spark.sql.functions.arrays_zip
zipArraysDf: org.apache.spark.sql.DataFrame = [game: string, release: string ... 2 more fields]
finalDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [game: string, release: string ... 3 more fields]


In [8]:
finalDf.printSchema()
finalDf.show(false)

root
 |-- game: string (nullable = true)
 |-- release: string (nullable = true)
 |-- class: string (nullable = true)
 |-- subclass: string (nullable = true)
 |-- super: string (nullable = true)

+---------+-------+-------+------------+---------------+
|game     |release|class  |subclass    |super          |
+---------+-------+-------+------------+---------------+
|Destiny 1|d1     |Hunter |Bladedancer |Arc Blade      |
|Destiny 2|d2     |Hunter |Gunslinger  |Golden Gun     |
|Destiny 2|d2     |Hunter |Arcstrider  |Arc Staff      |
|Destiny 2|d2     |Hunter |Nightstalker|Shadowshot     |
|Destiny 1|d1     |Titan  |Defender    |Ward of Dawn   |
|Destiny 2|d2     |Titan  |Striker     |Fist of Havoc  |
|Destiny 2|d2     |Titan  |Sentinel    |Sentinel Shield|
|Destiny 2|d2     |Titan  |Sunbreaker  |Hammer of Sol  |
|Destiny 1|d1     |Warlock|Sunsigner   |Radiance       |
|Destiny 2|d2     |Warlock|Dawnblade   |Daybreak       |
|Destiny 2|d2     |Warlock|Voidwalker  |Nova Bomb      |
|Destin

### pivots

In [9]:
spark.conf.set("spark.sql.shuffle.partitions", 8) // set to something small for testing

In [10]:
val noAggPivotDf = (finalDf
                  .groupBy($"game")
                  .pivot($"class", Seq("Hunter", "Titan", "Warlock"))
                  .agg(expr("first(super)"))
                   )

noAggPivotDf: org.apache.spark.sql.DataFrame = [game: string, Hunter: string ... 2 more fields]


In [11]:
noAggPivotDf.show()

+---------+----------+-------------+--------+
|     game|    Hunter|        Titan| Warlock|
+---------+----------+-------------+--------+
|Destiny 2|Golden Gun|Hammer of Sol|Daybreak|
|Destiny 1| Arc Blade| Ward of Dawn|Radiance|
+---------+----------+-------------+--------+



In [12]:
val aggPivotDf = (finalDf
                    .groupBy($"class")
                    .pivot($"game", Seq("Destiny 1", "Destiny 2"))
                  
                    .agg(expr("count(subclass)"))
                 )

aggPivotDf: org.apache.spark.sql.DataFrame = [class: string, Destiny 1: bigint ... 1 more field]


In [13]:
aggPivotDf.show()

+-------+---------+---------+
|  class|Destiny 1|Destiny 2|
+-------+---------+---------+
| Hunter|        1|        3|
|  Titan|        1|        3|
|Warlock|        1|        3|
+-------+---------+---------+



### write to parquet

In [11]:
noAggPivotDf.write.format("parquet").mode("overwrite").save("./table.parquet")

In [15]:
!ls -ltr ./table.parquet

total 8
-rw-r--r--. 1 ubuntu ubuntu 1269 Nov  7 18:48 part-00000-324c54fa-63d3-4b98-87d0-4c392a4c4716-c000.snappy.parquet
-rw-r--r--. 1 ubuntu ubuntu 1251 Nov  7 18:48 part-00005-324c54fa-63d3-4b98-87d0-4c392a4c4716-c000.snappy.parquet
-rw-r--r--. 1 ubuntu ubuntu    0 Nov  7 18:48 _SUCCESS



### read back in

In [16]:
val readDf = spark.read.parquet("./table.parquet")

readDf: org.apache.spark.sql.DataFrame = [game: string, Hunter: string ... 2 more fields]


In [17]:
readDf.printSchema()
readDf.show()
println(readDf.getClass)

root
 |-- game: string (nullable = true)
 |-- Hunter: string (nullable = true)
 |-- Titan: string (nullable = true)
 |-- Warlock: string (nullable = true)

+---------+----------+-------------+--------+
|     game|    Hunter|        Titan| Warlock|
+---------+----------+-------------+--------+
|Destiny 2|Golden Gun|Hammer of Sol|Daybreak|
|Destiny 1| Arc Blade| Ward of Dawn|Radiance|
+---------+----------+-------------+--------+

class org.apache.spark.sql.Dataset


### test python api

In [18]:
%%python
read_df = spark.read.parquet("./table.parquet")

In [19]:
%%python
read_df.printSchema()
read_df.show()
print(type(read_df))

root
 |-- game: string (nullable = true)
 |-- Hunter: string (nullable = true)
 |-- Titan: string (nullable = true)
 |-- Warlock: string (nullable = true)

+---------+----------+-------------+--------+
|     game|    Hunter|        Titan| Warlock|
+---------+----------+-------------+--------+
|Destiny 2|Golden Gun|Hammer of Sol|Daybreak|
|Destiny 1| Arc Blade| Ward of Dawn|Radiance|
+---------+----------+-------------+--------+

<class 'pyspark.sql.dataframe.DataFrame'>


### stop session

In [20]:
spark.stop()

#### garrett r peternel